In [43]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pprint
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from IPython.core.display import display, HTML
import unicodedata
from selenium.webdriver import Firefox
import scrapy
import logging
import json
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
from scrapy.loader import ItemLoader

## Nivell 1

- Exercici 1

    Realitza web scraping d'una pàgina de la borsa de Madrid (https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium.

In [2]:
# First with BeautifulSoup
# URL with data of prices
url_prices = 'https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'

# Make a request of the URL to put hmtl code inside
# I used before requests library I think is vey usefull
url_prices_get = requests.get(url_prices)

In [3]:
# Use of BeautifulSoup library
url_prices_get_hmtl = BeautifulSoup(url_prices_get.content, 'html.parser')
display(HTML(str(url_prices_get_hmtl.prettify())))

In [4]:
# Find the table on the html code
url_prices_get_hmtl_table = url_prices_get_hmtl.find("table", attrs={"id": "ctl00_Contenido_tblAcciones"})
display(HTML(str(url_prices_get_hmtl_table)))

Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
ACCIONA,"127,2000","1,84","127,6000","125,3000",117.279,"14.845,10",09/07/2021,Cierre
ACERINOX,"10,6200","3,76","10,6400","10,2250",796.533,"8.367,00",09/07/2021,Cierre
ACS,"22,1200","2,08","22,2300","21,7200",621.059,"13.713,34",09/07/2021,Cierre
AENA,"138,6500","-0,25","140,0000","137,5000",130.834,"18.156,46",09/07/2021,Cierre
ALMIRALL,"14,5400","0,14","14,7800","14,3800",332.159,"4.826,03",09/07/2021,Cierre
AMADEUS,"57,5800","1,02","58,2800","56,3800",796.553,"45.863,49",09/07/2021,Cierre
ARCELORMIT.,"26,3000","5,41","26,3150","25,0400",595.144,"15.381,65",09/07/2021,Cierre
B.SANTANDER,"3,1295","2,35","3,1390","3,0400",35.674.970,"110.794,23",09/07/2021,Cierre
BA.SABADELL,"0,5676","4,84","0,5686","0,5428",24.954.670,"13.968,68",09/07/2021,Cierre
BANKINTER,"4,2470","3,41","4,2560","4,1180",1.856.812,"7.838,09",09/07/2021,Cierre


In [5]:
# Data of the table
url_prices_get_hmtl_table_info = url_prices_get_hmtl_table.find_all("tr")
display(HTML(str(url_prices_get_hmtl_table_info)))

Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
ACCIONA,"127,2000","1,84","127,6000","125,3000",117.279,"14.845,10",09/07/2021,Cierre
ACERINOX,"10,6200","3,76","10,6400","10,2250",796.533,"8.367,00",09/07/2021,Cierre
ACS,"22,1200","2,08","22,2300","21,7200",621.059,"13.713,34",09/07/2021,Cierre
AENA,"138,6500","-0,25","140,0000","137,5000",130.834,"18.156,46",09/07/2021,Cierre
ALMIRALL,"14,5400","0,14","14,7800","14,3800",332.159,"4.826,03",09/07/2021,Cierre
AMADEUS,"57,5800","1,02","58,2800","56,3800",796.553,"45.863,49",09/07/2021,Cierre
ARCELORMIT.,"26,3000","5,41","26,3150","25,0400",595.144,"15.381,65",09/07/2021,Cierre
B.SANTANDER,"3,1295","2,35","3,1390","3,0400",35.674.970,"110.794,23",09/07/2021,Cierre
BA.SABADELL,"0,5676","4,84","0,5686","0,5428",24.954.670,"13.968,68",09/07/2021,Cierre
BANKINTER,"4,2470","3,41","4,2560","4,1180",1.856.812,"7.838,09",09/07/2021,Cierre


In [6]:
# Delete data table heading
url_prices_get_hmtl_table_info_columns = []

# Find columns names and clean them (blanquets, %, non unicode characters, ...)
# th is on heading so we use it to detect columns
for colum in url_prices_get_hmtl_table_info[0].find_all("th"):
    url_prices_get_hmtl_table_info_columns.append(unicodedata.normalize("NFKD",colum.text.replace('\n', ' ').replace('.', '').replace('(', '').replace(')', '').replace(' ', '').replace('%', 'pctg').strip().lower().encode("ascii","ignore").decode("ascii")))

# Columns of table
print(url_prices_get_hmtl_table_info_columns)

['nombre', 'lt', 'pctgdif', 'mx', 'mn', 'volumen', 'efectivomiles', 'fecha', 'hora']


In [7]:
# Dataframe creation
table = pd.DataFrame(columns = url_prices_get_hmtl_table_info_columns)
display(table)

,nombre,lt,pctgdif,mx,mn,volumen,efectivomiles,fecha,hora


In [8]:
# Fill dataframe with table's rows and columns
table_data = []
# Every row
for row in range((len(url_prices_get_hmtl_table_info)-1)): 
    # And on every column on that row
    for column in url_prices_get_hmtl_table_info[row+1].find_all("td"):
        table_data.append(column.text.replace('\n', ' ').strip())
    table.loc[row] = table_data
    table_data = []

In [9]:
# Finally let's see table dataframe
display(table)

,nombre,lt,pctgdif,mx,mn,volumen,efectivomiles,fecha,hora
0,ACCIONA,"127,2000","1,84","127,6000","125,3000",117.279,"14.845,10",09/07/2021,Cierre
1,ACERINOX,"10,6200","3,76","10,6400","10,2250",796.533,"8.367,00",09/07/2021,Cierre
2,ACS,"22,1200","2,08","22,2300","21,7200",621.059,"13.713,34",09/07/2021,Cierre
3,AENA,"138,6500","-0,25","140,0000","137,5000",130.834,"18.156,46",09/07/2021,Cierre
4,ALMIRALL,"14,5400","0,14","14,7800","14,3800",332.159,"4.826,03",09/07/2021,Cierre
5,AMADEUS,"57,5800","1,02","58,2800","56,3800",796.553,"45.863,49",09/07/2021,Cierre
6,ARCELORMIT.,"26,3000","5,41","26,3150","25,0400",595.144,"15.381,65",09/07/2021,Cierre
7,B.SANTANDER,"3,1295","2,35","3,1390","3,0400",35.674.970,"110.794,23",09/07/2021,Cierre
8,BA.SABADELL,"0,5676","4,84","0,5686","0,5428",24.954.670,"13.968,68",09/07/2021,Cierre
9,BANKINTER,"4,2470","3,41","4,2560","4,1180",1.856.812,"7.838,09",09/07/2021,Cierre


In [10]:
# Now working with Selenium

# Path to binary donloaded
path = "H:\\_vertex\\_formacio\\_python - cibernarium\\_16\\_16_13\\Chromedriver.exe"

driver = webdriver.Chrome(path)

# Web to investigate
web = "https://www.bolsamadrid.es/esp/aspx/Mercados/PreciosIGBM.aspx"
driver.get(web)

In [11]:
# Display URL
display(web)
# or
display(driver.current_url)

# Display web's title
display(driver.title)

'https://www.bolsamadrid.es/esp/aspx/Mercados/PreciosIGBM.aspx'

'https://www.bolsamadrid.es/esp/aspx/Mercados/PreciosIGBM.aspx'

'Bolsa de Madrid - Precios de la sesión por sectores IGBM: Petróleo y Energía'

In [12]:
# Find table
web_table = driver.find_element_by_xpath('//*[@id="ctl00_Contenido_tblAcciones"]/tbody')
display(web_table)
display(HTML(str(web_table)))

<selenium.webdriver.remote.webelement.WebElement (session="0dac26d1c540a32af52692977df2e42e", element="55548679-46d4-4453-8fef-d719347f5342")>

In [13]:
# Delete data table heading
web_table_columns = []

# Find columns names and clean them (blanquets, %, non unicode characters, ...)
# tr is on heading so we use it to detect rows
# th is on heading so we use it to detect columns
for row in web_table.find_elements_by_xpath('.//tr'):
    for column in row.find_elements_by_xpath('./th'):
        web_table_columns.append(unicodedata.normalize("NFKD",column.get_attribute('textContent').replace('\n', ' ').replace('.', '').replace('(', '').replace(')', '').replace(' ', '').replace('%', 'pctg').strip().lower().encode("ascii","ignore").decode("ascii")))

# Columns of table
print(web_table_columns)

['nombre', 'lt', 'pctgdif', 'mx', 'mn', 'volumen', 'efectivomiles', 'fecha', 'hora']


In [14]:
# Dataframe creation
table_selenium = pd.DataFrame(columns = web_table_columns)
display(table_selenium)

,nombre,lt,pctgdif,mx,mn,volumen,efectivomiles,fecha,hora


In [15]:
# Fill dataframe with table's rows and columns
web_table_data = []
row_table = 0

for row in web_table.find_elements_by_xpath('.//tr'):
    for column in row.find_elements_by_xpath('./td'):
        web_table_data.append(column.get_attribute('textContent').strip())
    if web_table_data == []:
        continue
    # Not take into account subtitle's group
    if (web_table_data[5]=='') is False:
        table_selenium.loc[row_table] = web_table_data
        row_table = row_table + 1
    web_table_data = []

In [16]:
# Finally let's see the table/dataframe
display(table_selenium)

,nombre,lt,pctgdif,mx,mn,volumen,efectivomiles,fecha,hora
0,REPSOL,"9,8480","1,20","9,8770","9,6010",6.268.091,"61.468,66",09/07/2021,Cierre
1,ENAGAS,"18,3300","0,63","18,4500","18,2650",799.971,"14.671,54",09/07/2021,Cierre
2,ENDESA,"21,2500","0,57","21,2800","21,0400",3.384.747,"71.678,84",09/07/2021,Cierre
3,IBERDROLA,"10,2800","0,69","10,3000","10,1750",12.084.486,"123.849,73",09/07/2021,Cierre
4,NATURGY,"22,0700","0,68","22,1300","21,8700",636.295,"14.028,14",09/07/2021,Cierre
5,R.E.C.,"15,7500","0,80","15,8000","15,6000",1.579.491,"24.839,13",09/07/2021,Cierre
6,AUDAX RENOV.,"1,9230","-0,36","1,9480","1,9160",531.910,"1.025,82",09/07/2021,Cierre
7,ECOENER,"5,5600","-1,07","5,6500","5,5600",11.400,"63,79",09/07/2021,Cierre
8,GRENERGY,"29,7000","0,51","30,0000","29,1000",33.712,"990,67",09/07/2021,Cierre
9,SOLARIA,"16,9000","-2,23","17,2900","16,6450",812.468,"13.675,60",09/07/2021,Cierre


In [17]:
# Close browser
driver.close()

## Nivell 2

- Exercici 2

    Documenta en un word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.

<html>

<head>
<meta http-equiv=Content-Type content="text/html; charset=windows-1252">
<meta name=Generator content="Microsoft Word 15 (filtered)">
<style>
<!--
 /* Font Definitions */
 @font-face
	{font-family:Wingdings;
	panose-1:5 0 0 0 0 0 0 0 0 0;}
@font-face
	{font-family:"Cambria Math";
	panose-1:2 4 5 3 5 4 6 3 2 4;}
@font-face
	{font-family:Calibri;
	panose-1:2 15 5 2 2 2 4 3 2 4;}
@font-face
	{font-family:"Open Sans";
	panose-1:2 11 6 6 3 5 4 2 2 4;}
 /* Style Definitions */
 p.MsoNormal, li.MsoNormal, div.MsoNormal
	{margin-top:0cm;
	margin-right:0cm;
	margin-bottom:8.0pt;
	margin-left:0cm;
	line-height:107%;
	font-size:11.0pt;
	font-family:"Calibri",sans-serif;}
a:link, span.MsoHyperlink
	{color:blue;
	text-decoration:underline;}
p.MsoListParagraph, li.MsoListParagraph, div.MsoListParagraph
	{margin-top:0cm;
	margin-right:0cm;
	margin-bottom:8.0pt;
	margin-left:36.0pt;
	line-height:107%;
	font-size:11.0pt;
	font-family:"Calibri",sans-serif;}
p.MsoListParagraphCxSpFirst, li.MsoListParagraphCxSpFirst, div.MsoListParagraphCxSpFirst
	{margin-top:0cm;
	margin-right:0cm;
	margin-bottom:0cm;
	margin-left:36.0pt;
	line-height:107%;
	font-size:11.0pt;
	font-family:"Calibri",sans-serif;}
p.MsoListParagraphCxSpMiddle, li.MsoListParagraphCxSpMiddle, div.MsoListParagraphCxSpMiddle
	{margin-top:0cm;
	margin-right:0cm;
	margin-bottom:0cm;
	margin-left:36.0pt;
	line-height:107%;
	font-size:11.0pt;
	font-family:"Calibri",sans-serif;}
p.MsoListParagraphCxSpLast, li.MsoListParagraphCxSpLast, div.MsoListParagraphCxSpLast
	{margin-top:0cm;
	margin-right:0cm;
	margin-bottom:8.0pt;
	margin-left:36.0pt;
	line-height:107%;
	font-size:11.0pt;
	font-family:"Calibri",sans-serif;}
span.jlqj4b
	{mso-style-name:jlqj4b;}
.MsoChpDefault
	{font-family:"Calibri",sans-serif;}
.MsoPapDefault
	{margin-bottom:8.0pt;
	line-height:107%;}
@page WordSection1
	{size:595.3pt 841.9pt;
	margin:35.45pt 35.35pt 70.85pt 1.0cm;}
div.WordSection1
	{page:WordSection1;}
 /* List Definitions */
 ol
	{margin-bottom:0cm;}
ul
	{margin-bottom:0cm;}
-->
</style>

</head>

<body lang=ES link=blue vlink="#954F72" style='word-wrap:break-word'>

<div class=WordSection1>

<p class=MsoNormal style='margin-bottom:0cm;line-height:normal'><b><span
lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif;
color:#1F3864'>Madrid Stock Exchange Dataset</span></b></p>

<p class=MsoNormal style='margin-bottom:0cm;line-height:normal'><b><span
lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif;
color:#1F3864'>35+ Stock exchange, shares, money, percentage, opening details
and more</span></b></p>

<p class=MsoNormal style='margin-bottom:0cm;line-height:normal'><span
lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif;
color:#1F3864'>&nbsp;</span></p>

<p class=MsoNormal style='margin-bottom:0cm;line-height:normal'><b><span
lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif;
color:#1F3864'>Antoni Riba  </span></b><span lang=EN-GB style='font-size:10.0pt;
font-family:"Open Sans",sans-serif;color:#1F3864'>• &nbsp;updated 3 weeks (Version
1)</span></p>

<p class=MsoNormal style='margin-bottom:0cm;line-height:normal'><span
lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif;
color:#0070C0'>&nbsp;</span></p>

<p class=MsoNormal style='margin-bottom:0cm;line-height:normal'><span
lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif;
color:#0070C0'>Data</span></p>

<p class=MsoNormal style='margin-bottom:0cm;line-height:normal'><span
lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif;
color:#0070C0'>_____</span></p>

<p class=MsoNormal style='margin-bottom:0cm;line-height:normal'><span
lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif'>&nbsp;</span></p>

<p class=MsoNormal style='margin-bottom:0cm;line-height:normal'><b><span
lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif'>Usability 
</span></b><span lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif'> </span><span
lang=EN-GB style='font-size:9.0pt;font-family:"Open Sans",sans-serif;
color:#A6A6A6'>8.2</span><span lang=EN-GB style='font-size:10.0pt;font-family:
"Open Sans",sans-serif;color:#A6A6A6'> </span><span lang=EN-GB
style='font-size:10.0pt;font-family:"Open Sans",sans-serif'>  <b>License  </b> </span><span
lang=EN-GB style='font-size:9.0pt;font-family:"Open Sans",sans-serif;
color:#A6A6A6'>CC0: Public Domain  </span><b><span lang=EN-GB style='font-size:
10.0pt;font-family:"Open Sans",sans-serif'>Tags  </span></b><span lang=EN-GB
style='font-size:10.0pt;font-family:"Open Sans",sans-serif'> </span><span
lang=EN-GB style='font-size:9.0pt;font-family:"Open Sans",sans-serif;
color:#A6A6A6'>stock exchange</span><span lang=EN-GB style='font-size:9.0pt;
font-family:"Open Sans",sans-serif;color:black'>,</span><span lang=EN-GB
style='font-size:9.0pt;font-family:"Open Sans",sans-serif;color:#A6A6A6'> money</span><span
lang=EN-GB style='font-size:9.0pt;font-family:"Open Sans",sans-serif;
color:black'>, </span><span lang=EN-GB style='font-size:9.0pt;font-family:"Open Sans",sans-serif;
color:#A6A6A6'>shares</span><span lang=EN-GB style='font-size:9.0pt;font-family:
"Open Sans",sans-serif;color:black'>, </span><span lang=EN-GB style='font-size:
9.0pt;font-family:"Open Sans",sans-serif;color:#A6A6A6'>euros</span></p>

<p class=MsoNormal style='margin-bottom:0cm;line-height:normal'><span
lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif'>&nbsp;</span></p>

<p class=MsoNormal style='margin-bottom:0cm;line-height:normal'><b><span
lang=EN-GB style='font-size:12.0pt;font-family:"Open Sans",sans-serif;
color:#A6A6A6'>Description</span></b></p>

<p class=MsoNormal style='line-height:normal'><b><span lang=EN-GB
style='font-size:10.0pt;font-family:"Open Sans",sans-serif'>General Info</span></b></p>

<p class=MsoNormal style='text-align:justify;line-height:normal'><span
lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif'>The
Madrid Stock Exchange is the main stock market in Spain. It is based in the
Palacio de la Bolsa in Madrid. Its reference index is the &quot;General Index
of the Madrid Stock Exchange&quot; IGBM. It is part of the Spanish Stock
Exchange and Markets Society.</span></p>

<p class=MsoNormal style='text-align:justify;line-height:normal'><span
lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif'>This is
a set of just over 35 companies quotes on Madrid Stock Exchange collected from </span><a
href="https://www.bolsamadrid.es"><span lang=EN-GB style='font-size:10.0pt;
font-family:"Open Sans",sans-serif'>https://www.bolsamadrid.es</span></a><span
lang=EN-GB style='font-size:10.0pt;font-family:"Open Sans",sans-serif'> web. I
will also upload more data in the future as I collect them. This set contains
the:</span></p>

<p class=MsoListParagraphCxSpFirst style='margin-bottom:10.0pt;text-indent:
-18.0pt;line-height:normal;text-autospace:none'><span lang=es style='font-size:
10.0pt;font-family:Symbol'>·<span style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><b><i><span lang=es style='font-size:12.0pt;color:#A6A6A6'>nombre</span></i></b><span
lang=es style='font-size:12.0pt'>;                         </span><span
class=jlqj4b><span lang=EN>company name</span></span></p>

<p class=MsoListParagraphCxSpMiddle style='margin-bottom:10.0pt;text-indent:
-18.0pt;line-height:normal;text-autospace:none'><span lang=es style='font-size:
10.0pt;font-family:Symbol'>·<span style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><b><i><span lang=es style='font-size:12.0pt;color:#A6A6A6'>lt</span></i></b><span
lang=es style='font-size:12.0pt'>;                                      last </span><span
class=jlqj4b><span lang=EN>value shares</span></span></p>

<p class=MsoListParagraphCxSpMiddle style='margin-bottom:10.0pt;text-indent:
-18.0pt;line-height:normal;text-autospace:none'><span lang=es style='font-size:
10.0pt;font-family:Symbol'>·<span style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><b><i><span lang=es style='font-size:12.0pt;color:#A6A6A6'>pctgdif</span></i></b><span
lang=es style='font-size:12.0pt'>;                           </span><span
class=jlqj4b><span lang=EN>percentage difference of the value share compared
with the previous day</span></span></p>

<p class=MsoListParagraphCxSpMiddle style='margin-bottom:10.0pt;text-indent:
-18.0pt;line-height:normal;text-autospace:none'><span lang=es style='font-size:
10.0pt;font-family:Symbol'>·<span style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><b><i><span lang=es style='font-size:12.0pt;color:#A6A6A6'>mx</span></i></b><span
lang=es style='font-size:12.0pt'>;                                   </span><span
class=jlqj4b><span lang=EN>maximum value shares</span></span></p>

<p class=MsoListParagraphCxSpMiddle style='margin-bottom:10.0pt;text-indent:
-18.0pt;line-height:normal;text-autospace:none'><span lang=es style='font-size:
10.0pt;font-family:Symbol'>·<span style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><b><i><span lang=es style='font-size:12.0pt;color:#A6A6A6'>mn</span></i></b><span
lang=es style='font-size:12.0pt'>;                                   </span><span
class=jlqj4b><span lang=EN>minimum value shares</span></span></p>

<p class=MsoListParagraphCxSpMiddle style='margin-bottom:10.0pt;text-indent:
-18.0pt;line-height:normal;text-autospace:none'><span lang=es style='font-size:
10.0pt;font-family:Symbol'>·<span style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><b><i><span lang=es style='font-size:12.0pt;color:#A6A6A6'>volumen</span></i></b><span
lang=es style='font-size:12.0pt'>;                        </span><span
class=jlqj4b><span lang=EN>number of shares</span></span></p>

<p class=MsoListParagraphCxSpMiddle style='margin-bottom:10.0pt;text-indent:
-18.0pt;line-height:normal;text-autospace:none'><span lang=es style='font-size:
10.0pt;font-family:Symbol'>·<span style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><b><i><span lang=es style='font-size:12.0pt;color:#A6A6A6'>efectivomiles</span></i></b><span
lang=es style='font-size:12.0pt'>;               </span><span class=jlqj4b><span
lang=EN>total price of the company's shares [in thousands of euros]</span></span></p>

<p class=MsoListParagraphCxSpMiddle style='margin-bottom:10.0pt;text-indent:
-18.0pt;line-height:normal;text-autospace:none'><span lang=es style='font-size:
10.0pt;font-family:Symbol'>·<span style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><b><i><span lang=es style='font-size:12.0pt;color:#A6A6A6'>fecha</span></i></b><span
lang=es style='font-size:12.0pt'>;                              </span><span
class=jlqj4b><span lang=EN>date of last change in stock value shares</span></span></p>

<p class=MsoListParagraphCxSpLast style='margin-bottom:10.0pt;text-indent:-18.0pt;
line-height:normal;text-autospace:none'><span lang=es style='font-size:10.0pt;
font-family:Symbol'>·<span style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><b><i><span lang=es style='font-size:12.0pt;color:#A6A6A6'>hora</span></i></b><span
lang=es style='font-size:12.0pt'>;                                </span><span
class=jlqj4b><span lang=EN>time of last change in stock value shares</span></span></p>

<p class=MsoNormal style='line-height:normal'><b><span lang=EN-GB
style='font-size:10.0pt;font-family:"Open Sans",sans-serif'>Possible Uses</span></b></p>

<p class=MsoNormal style='line-height:normal'><span lang=EN-GB
style='font-size:10.0pt;font-family:"Open Sans",sans-serif'>Evaluate possible
financial investments.</span></p>

<p class=MsoNormal><span lang=EN-GB style='font-size:10.0pt;line-height:107%;
font-family:"Open Sans",sans-serif'>&nbsp;</span></p>

</div>

</body>

</html>


## Nivell 3

- Exercici 3

    Tria una página web que tu vulguis i realitza web scraping mitjançant la llibreria Scrapy.